In [ ]:
# notebooks/time_series_analysis.ipynb
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Time Series Analysis of Financial Transactions\n",
    "\n",
    "This notebook demonstrates the time series preprocessing and analysis capabilities of the Financial Health Assistant."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import os\n",
    "import sys\n",
    "\n",
    "# Add parent directory to path for imports\n",
    "sys.path.append(os.path.dirname(os.getcwd()))\n",
    "\n",
    "from app.models.time_series.time_series_processor import TimeSeriesProcessor\n",
    "\n",
    "# Set plot style\n",
    "plt.style.use('ggplot')\n",
    "sns.set(font_scale=1.2)\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Transaction Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load transaction data\n",
    "transactions_path = '../app/data/processed/transactions_clean.csv'\n",
    "\n",
    "if os.path.exists(transactions_path):\n",
    "    df = pd.read_csv(transactions_path)\n",
    "    print(f\"Loaded {len(df)} transactions\")\n",
    "else:\n",
    "    print(f\"Error: {transactions_path} not found\")\n",
    "    # Create sample data for demonstration if necessary\n",
    "    from datetime import datetime, timedelta\n",
    "    \n",
    "    # Generate dates\n",
    "    base_date = datetime(2023, 1, 1)\n",
    "    dates = [base_date + timedelta(days=i) for i in range(100)]\n",
    "    \n",
    "    # Generate random transactions\n",
    "    np.random.seed(42)\n",
    "    n_transactions = 300\n",
    "    transaction_dates = np.random.choice(dates, size=n_transactions)\n",
    "    \n",
    "    # Generate random categories\n",
    "    categories = ['food', 'transport', 'shopping', 'utilities', 'entertainment']\n",
    "    transaction_categories = np.random.choice(categories, size=n_transactions)\n",
    "    \n",
    "    # Generate random amounts (negative for expenses)\n",
    "    amounts = -np.random.uniform(10, 200, size=n_transactions)\n",
    "    \n",
    "    # Add some income transactions (positive amounts)\n",
    "    income_indices = np.random.choice(range(n_transactions), size=30)\n",
    "    amounts[income_indices] = np.random.uniform(500, 2000, size=30)\n",
    "    \n",
    "    # Create DataFrame\n",
    "    df = pd.DataFrame({\n",
    "        'transaction_date': transaction_dates,\n",
    "        'category': transaction_categories,\n",
    "        'amount': amounts\n",
    "    })\n",
    "    print(f\"Created sample data with {len(df)} transactions\")\n",
    "\n",
    "# Display sample\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Preprocess Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Map columns if needed\n",
    "column_mapping = {\n",
    "    'DATE': 'transaction_date',\n",
    "    ' WITHDRAWAL AMT ': 'withdrawal',\n",
    "    ' DEPOSIT AMT ': 'deposit',\n",
    "    'TRANSACTION DETAILS': 'description'\n",
    "}\n",
    "\n",
    "for old_col, new_col in column_mapping.items():\n",
    "    if old_col in df.columns and new_col not in df.columns:\n",
    "        df[new_col] = df[old_col]\n",
    "\n",
    "# Ensure date column is datetime\n",
    "if 'transaction_date' in df.columns:\n",
    "    df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')\n",
    "    df = df.dropna(subset=['transaction_date'])\n",
    "    print(f\"Date range: {df['transaction_date'].min()} to {df['transaction_date'].max()}\")\n",
    "\n",
    "# Create amount column if it doesn't exist\n",
    "if 'amount' not in df.columns and 'withdrawal' in df.columns and 'deposit' in df.columns:\n",
    "    df['withdrawal'] = pd.to_numeric(df['withdrawal'], errors='coerce').fillna(0)\n",
    "    df['deposit'] = pd.to_numeric(df['deposit'], errors='coerce').fillna(0)\n",
    "    df['amount'] = df['deposit'] - df['withdrawal']\n",
    "    print(\"Created 'amount' column from withdrawal and deposit columns\")\n",
    "\n",
    "# Initialize time series processor\n",
    "processor = TimeSeriesProcessor()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Aggregate Transactions to Time Series Format"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create different frequency time series\n",
    "daily_ts = processor.convert_to_time_series(\n",
    "    df,\n",
    "    date_col='transaction_date',\n",
    "    amount_col='amount',\n",
    "    category_col='category',\n",
    "    freq='D'\n",
    ")\n",
    "print(f\"Created daily time series with {len(daily_ts)} days\")\n",
    "\n",
    "weekly_ts = processor.convert_to_time_series(\n",
    "    df,\n",
    "    date_col='transaction_date',\n",
    "    amount_col='amount',\n",
    "    category_col='category',\n",
    "    freq='W'\n",
    ")\n",
    "print(f\"Created weekly time series with {len(weekly_ts)} weeks\")\n",
    "\n",
    "monthly_ts = processor.convert_to_time_series(\n",
    "    df,\n",
    "    date_col='transaction_date',\n",
    "    amount_col='amount',\n",
    "    category_col='category',\n",
    "    freq='M'\n",
    ")\n",
    "print(f\"Created monthly time series with {len(monthly_ts)} months\")\n",
    "\n",
    "# Display monthly time series\n",
    "monthly_ts.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Visualize Time Series Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot monthly spending\n",
    "plt.figure(figsize=(12, 6))\n",
    "\n",
    "if 'category' in monthly_ts.columns:\n",
    "    # Create pivot table for better visualization\n",
    "    pivot_df = monthly_ts.pivot(index='transaction_date', columns='category', values='amount_sum')\n",
    "    pivot_df.plot(kind='bar', stacked=True, ax=plt.gca())\n",
    "else:\n",
    "    monthly_ts.plot(x='transaction_date', y='amount_sum', kind='bar', ax=plt.gca())\n",
    "\n",
    "plt.title('Monthly Total Spending')\n",
    "plt.xlabel('Month')\n",
    "plt.ylabel('Amount')\n",
    "plt.xticks(rotation=45)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Extract Temporal Features"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Add temporal features to daily time series\n",
    "daily_features = processor.extract_temporal_features(daily_ts)\n",
    "print(f\"Added {len(daily_features.columns) - len(daily_ts.columns)} temporal features\")\n",
    "\n",
    "# Display new columns\n",
    "new_cols = [col for col in daily_features.columns if col not in daily_ts.columns]\n",
    "print(\"New features:\", new_cols)\n",
    "\n",
    "# Display sample of data with features\n",
    "daily_features[['transaction_date', 'amount_sum'] + new_cols].head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Analyze Day of Week Patterns"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze spending by day of week\n",
    "if 'dayofweek' in daily_features.columns:\n",
    "    plt.figure(figsize=(12, 6))\n",
    "    \n",
    "    # Create day of week labels\n",
    "    day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']\n",
    "    \n",
    "    if 'category' in daily_features.columns:\n",
    "        # Group by day of week and category\n",
    "        dow_spending = daily_features.groupby(['dayofweek', 'category'])['amount_sum'].mean().reset_index()\n",
    "        \n",
    "        # Create pivot table\n",
    "        pivot_df = dow_spending.pivot(index='dayofweek', columns='category', values='amount_sum')\n",
    "        \n",
    "        # Plot stacked bar chart\n",
    "        pivot_df.plot(kind='bar', stacked=True, ax=plt.gca())\n",
    "    else:\n",
    "        # Group by day of week only\n",
    "        dow_spending = daily_features.groupby('dayofweek')['amount_sum'].mean()\n",
    "        dow_spending.plot(kind='bar', ax=plt.gca())\n",
    "    \n",
    "    plt.title('Average Spending by Day of Week')\n",
    "    plt.xlabel('Day of Week')\n",
    "    plt.ylabel('Average Amount')\n",
    "    plt.xticks(range(7), day_names, rotation=45)\n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "else:\n",
    "    print(\"Day of week feature not available\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Create and Analyze Lagged Features"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Add lag features\n",
    "daily_with_lags = processor.create_lagged_features(\n",
    "    daily_ts,\n",
    "    value_col='amount_sum',\n",
    "    lag_periods=[1, 7, 30],\n",
    "    group_col='category' if 'category' in daily_ts.columns else None\n",
    ")\n",
    "\n",
    "# Display lag features\n",
    "lag_cols = [col for col in daily_with_lags.columns if '_lag_' in col]\n",
    "print(f\"Created {len(lag_cols)} lag features: {lag_cols}\")\n",
    "\n",
    "# Analyze correlation between current spending and lagged spending\n",
    "if lag_cols:\n",
    "    # Remove rows with NaN values (from lagging)\n",
    "    correlation_df = daily_with_lags.dropna(subset=['amount_sum'] + lag_cols)\n",
    "    \n",
    "    # Calculate correlations\n",
    "    correlations = correlation_df[['amount_sum'] + lag_cols].corr()['amount_sum']\n",
    "    \n",
    "    # Plot correlations\n",
    "    plt.figure(figsize=(10, 6))\n",
    "    correlations[1:].sort_values().plot(kind='bar')\n",
    "    plt.title('Correlation between Current and Lagged Spending')\n",
    "    plt.xlabel('Lag Feature')\n",
    "    plt.ylabel('Correlation')\n",
    "    plt.axhline(y=0, color='r', linestyle='-')\n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    print(\"Correlations:\")\n",
    "    print(correlations)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Create and Analyze Rolling Features"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Add rolling features\n",
    "daily_with_rolling = processor.create_rolling_features(\n",
    "    daily_ts,\n",
    "    value_col='amount_sum',\n",
    "    window_sizes=[7, 14, 30],\n",
    "    group_col='category' if 'category' in daily_ts.columns else None\n",
    ")\n",
    "\n",
    "# Display rolling features\n",
    "rolling_cols = [col for col in daily_with_rolling.columns if '_rolling_' in col]\n",
    "print(f\"Created {len(rolling_cols)} rolling features: {rolling_cols}\")\n",
    "\n",
    "# Plot original vs rolling mean\n",
    "if 'amount_sum_rolling_mean_7' in daily_with_rolling.columns:\n",
    "    plt.figure(figsize=(12, 6))\n",
    "    \n",
    "    # Sort by date for proper time series display\n",
    "    plot_df = daily_with_rolling.sort_values('transaction_date')\n",
    "    \n",
    "    # Plot for a specific category if available\n",
    "    if 'category' in plot_df.columns:\n",
    "        # Select a category for demonstration\n",
    "        example_category = plot_df['category'].value_counts().index[0]\n",
    "        plot_df = plot_df[plot_df['category'] == example_category]\n",
    "        title_suffix = f\" (Category: {example_category})\"\n",
    "    else:\n",
    "        title_suffix = \"\"\n",
    "    \n",
    "    # Plot original data and rolling means\n",
    "    plt.plot(plot_df['transaction_date'], plot_df['amount_sum'], 'o-', alpha=0.5, label='Daily')\n",
    "    plt.plot(plot_df['transaction_date'], plot_df['amount_sum_rolling_mean_7'], 'r-', label='7-day Rolling Mean')\n",
    "    \n",
    "    if 'amount_sum_rolling_mean_30' in plot_df.columns:\n",
    "        plt.plot(plot_df['transaction_date'], plot_df['amount_sum_rolling_mean_30'], 'g-', label='30-day Rolling Mean')\n",
    "    \n",
    "    plt.title(f'Daily vs. Rolling Average Spending{title_suffix}')\n",
    "    plt.xlabel('Date')\n",
    "    plt.ylabel('Amount')\n",
    "    plt.legend()\n",
    "    plt.xticks(rotation=45)\n",
    "    plt.tight_layout()\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. Detect and Analyze Outliers"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Detect outliers\n",
    "daily_with_outliers = processor.detect_outliers(\n",
    "    daily_ts,\n",
    "    value_col='amount_sum',\n",
    "    method='zscore',\n",
    "    threshold=2.5,  # Lower threshold to identify more outliers for demonstration\n",
    "    group_col='category' if 'category' in daily_ts.columns else None\n",
    ")\n",
    "\n",
    "# Count outliers\n",
    "outlier_col = 'amount_sum_is_outlier'\n",
    "if outlier_col in daily_with_outliers.columns:\n",
    "    num_outliers = daily_with_outliers[outlier_col].sum()\n",
    "    total_days = len(daily_with_outliers)\n",
    "    print(f\"Detected {num_outliers} outliers out of {total_days} days ({num_outliers/total_days:.1%})\")\n",
    "    \n",
    "    # View outliers\n",
    "    outliers = daily_with_outliers[daily_with_outliers[outlier_col] == 1]\n",
    "    print(\"\\nTop 5 outliers by amount:\")\n",
    "    display(outliers.sort_values('amount_sum', ascending=False).head(5)[['transaction_date', 'category', 'amount_sum']])\n",
    "    \n",
    "    # Plot outliers\n",
    "    plt.figure(figsize=(12, 6))\n",
    "    \n",
    "    # Sort by date\n",
    "    plot_df = daily_with_outliers.sort_values('transaction_date')\n",
    "    \n",
    "    # Plot normal points\n",
    "    normal = plot_df[plot_df[outlier_col] == 0]\n",
    "    plt.scatter(normal['transaction_date'], normal['amount_sum'], color='blue', alpha=0.5, label='Normal')\n",
    "    \n",
    "    # Plot outliers\n",
    "    outliers = plot_df[plot_df[outlier_col] == 1]\n",
    "    plt.scatter(outliers['transaction_date'], outliers['amount_sum'], color='red', alpha=0.8, label='Outlier')\n",
    "    \n",
    "    plt.title('Daily Spending with Outliers Highlighted')\n",
    "    plt.xlabel('Date')\n",
    "    plt.ylabel('Amount')\n",
    "    plt.legend()\n",
    "    plt.xticks(rotation=45)\n",
    "    plt.tight_layout()\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 10. Comprehensive Spending Pattern Visualization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate comprehensive spending pattern visualizations\n",
    "figures = processor.visualize_spending_patterns(\n",
    "    df,\n",
    "    date_col='transaction_date',\n",
    "    amount_col='amount',\n",
    "    category_col='category' if 'category' in df.columns else None,\n",
    "    freq='M'  # Monthly aggregation\n",
    ")\n",
    "\n",
    "print(f\"Created {len(figures)} visualizations\")\n",
    "print(f\"Visualization files saved to: {processor.visualization_dir}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 11. Save Processed Time Series Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Process full time series data with all features\n",
    "processed_daily = processor.process_time_series_data(\n",
    "    df,\n",
    "    date_col='transaction_date',\n",
    "    amount_col='amount',\n",
    "    category_col='category' if 'category' in df.columns else None,\n",
    "    freq='D'\n",
    ")\n",
    "\n",
    "processed_weekly = processor.process_time_series_data(\n",
    "    df,\n",
    "    date_col='transaction_date',\n",
    "    amount_col='amount',\n",
    "    category_col='category' if 'category' in df.columns else None,\n",
    "    freq='W'\n",
    ")\n",
    "\n",
    "processed_monthly = processor.process_time_series_data(\n",
    "    df,\n",
    "    date_col='transaction_date',\n",
    "    amount_col='amount',\n",
    "    category_col='category' if 'category' in df.columns else None,\n",
    "    freq='M'\n",
    ")\n",
    "\n",
    "# Create directory to save processed data\n",
    "os.makedirs('../app/data/processed/time_series', exist_ok=True)\n",
    "\n",
    "# Save processed data\n",
    "processed_daily.to_csv('../app/data/processed/time_series/daily_ts.csv', index=False)\n",
    "processed_weekly.to_csv('../app/data/processed/time_series/weekly_ts.csv', index=False)\n",
    "processed_monthly.to_csv('../app/data/processed/time_series/monthly_ts.csv', index=False)\n",
    "\n",
    "print(f\"Saved processed time series data to app/data/processed/time_series/\")\n",
    "print(f\"Daily time series has {len(processed_daily)} records with {len(processed_daily.columns)} features\")\n",
    "print(f\"Weekly time series has {len(processed_weekly)} records with {len(processed_weekly.columns)} features\")\n",
    "print(f\"Monthly time series has {len(processed_monthly)} records with {len(processed_monthly.columns)} features\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 12. Summary and Next Steps\n",
    "\n",
    "In this notebook, we've processed financial transaction data into time series format with various temporal features:\n",
    "\n",
    "1. Aggregated transactions to daily, weekly, and monthly frequencies\n",
    "2. Extracted temporal features (day of week, month, seasonality, etc.)\n",
    "3. Created lagged features to capture historical patterns\n",
    "4. Created rolling features to smooth out fluctuations\n",
    "5. Detected outliers in spending patterns\n",
    "6. Visualized spending patterns across different dimensions\n",
    "\n",
    "These processed time series datasets will be used in the next steps for:\n",
    "\n",
    "- Training forecasting models to predict future spending\n",
    "- Developing recommendation engine for personalized financial advice\n",
    "- Creating interactive visualizations for the web dashboard\n",
    "\n",
    "The time series features extracted here will help in identifying spending patterns, seasonal trends, and anomalies, which are crucial for accurate forecasting and meaningful recommendations."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}